In [1]:
%cd "C:\Users\hrish\George Mason University - O365 Production\DAEN 690 - Group Only-GRP - 063022"

C:\Users\hrish\George Mason University - O365 Production\DAEN 690 - Group Only-GRP - 063022


In [2]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from os import listdir
from keras.preprocessing import sequence

In [11]:
first_file = pd.read_csv('SimData_2022.06.30_10.27.04.csv.roar.csv',
                parse_dates = {'datetime': ['date', 'time']},
                low_memory=False)

second_file = pd.read_csv('SimData_2022.06.30_13.01.46.csv.roar.csv',
                parse_dates = {'datetime': ['date', 'time']},
                low_memory=False)

third_file = pd.read_csv('SimData_2022.06.30_13.05.32.csv.roar.csv',
                parse_dates = {'datetime': ['date', 'time']},
                low_memory=False)

fourth_file = pd.read_csv('SimData_2022.06.30_13.42.04.csv.roar.csv',
                parse_dates = {'datetime': ['date', 'time']},
                low_memory=False)

fifth_file = pd.read_csv('SimData_2022.07.07_09.56.57.csv.roar.csv',
                parse_dates = {'datetime': ['date', 'time']},
                low_memory=False)

# adding this file to balance hover or not
sixth_file = pd.read_csv('SimData_2022.07.07_14.32.46.csv.roar.csv',
                parse_dates = {'datetime': ['date', 'time']},
                low_memory=False)

df = pd.concat([first_file, second_file, third_file, fourth_file,fifth_file, sixth_file])

In [12]:
df.head(10)
df['turnDirection'] = df['trueHeading'].rolling(window = 5).apply(lambda x: x[-1]-x[0], raw=True)
df['Altdiff'] = df['absoluteAltitude'].rolling(window = 5).apply(lambda x: x[0]- x[-1], raw = True)
df['groundSpeedDiff'] = df['groundSpeed'].rolling(window = 5).apply(lambda x: x[0]- x[-1], raw = True)
df['turnDirection']  = df['turnDirection'].fillna(0)
df['Altdiff'] = df['Altdiff'].fillna(0)
df['groundSpeedDiff']=df['groundSpeedDiff'].fillna(0)
df.head(10)


,datetime,_id,latitude,longitude,gpsAltitude,absoluteAltitude,magneticHeading,trueHeading,pitch,roll,...,hasTransmissionChipCaution,hasTransmissionOilTemperatureWarning,hasTransmissionOilPressureWarning,timestamp,gpsDateTime,seconds,vil,turnDirection,Altdiff,groundSpeedDiff
0,2022-06-30 14:27:53.216,::1656613673.216,20.904799,-156.424679,36.82,0.12,204.071,213.554,1.594,0.665,...,False,False,False,2022-06-30T14:27:53.216Z,2022-06-30T14:27:53.216Z,1.656614e+09,-2,0.000,0.0,0.000000
1,2022-06-30 14:27:53.234,::1656613673.234,20.904799,-156.424679,36.82,0.12,204.064,213.547,1.593,0.683,...,False,False,False,2022-06-30T14:27:53.234Z,2022-06-30T14:27:53.234Z,1.656614e+09,-2,0.000,0.0,0.000000
2,2022-06-30 14:27:53.251,::1656613673.251,20.904799,-156.424679,36.82,0.12,204.058,213.541,1.593,0.707,...,False,False,False,2022-06-30T14:27:53.251Z,2022-06-30T14:27:53.251Z,1.656614e+09,-2,0.000,0.0,0.000000
3,2022-06-30 14:27:53.271,::1656613673.271,20.904799,-156.424679,36.81,0.12,204.051,213.534,1.594,0.735,...,False,False,False,2022-06-30T14:27:53.271Z,2022-06-30T14:27:53.271Z,1.656614e+09,-2,0.000,0.0,0.000000
4,2022-06-30 14:27:53.293,::1656613673.293,20.904799,-156.424679,36.81,0.12,204.043,213.526,1.596,0.770,...,False,False,False,2022-06-30T14:27:53.293Z,2022-06-30T14:27:53.293Z,1.656614e+09,-2,-0.028,0.0,-0.030217
5,2022-06-30 14:27:53.313,::1656613673.313,20.904799,-156.424679,36.81,0.12,204.035,213.518,1.600,0.807,...,False,False,False,2022-06-30T14:27:53.313Z,2022-06-30T14:27:53.313Z,1.656614e+09,-2,-0.029,0.0,-0.036142
6,2022-06-30 14:27:53.334,::1656613673.334,20.904799,-156.424679,36.81,0.12,204.029,213.512,1.605,0.838,...,False,False,False,2022-06-30T14:27:53.334Z,2022-06-30T14:27:53.334Z,1.656614e+09,-2,-0.029,0.0,-0.030809
7,2022-06-30 14:27:53.356,::1656613673.356,20.904799,-156.424679,36.82,0.12,204.023,213.506,1.612,0.868,...,False,False,False,2022-06-30T14:27:53.356Z,2022-06-30T14:27:53.356Z,1.656614e+09,-2,-0.028,0.0,-0.020737
8,2022-06-30 14:27:53.376,::1656613673.376,20.904799,-156.424679,36.82,0.12,204.017,213.499,1.622,0.893,...,False,False,False,2022-06-30T14:27:53.376Z,2022-06-30T14:27:53.376Z,1.656614e+09,-2,-0.027,0.0,-0.007110
9,2022-06-30 14:27:53.393,::1656613673.393,20.904799,-156.424679,36.82,0.12,204.012,213.495,1.633,0.909,...,False,False,False,2022-06-30T14:27:53.393Z,2022-06-30T14:27:53.393Z,1.656614e+09,-2,-0.023,0.0,0.004147


In [13]:
df2 = df[['datetime',
          'latitude',
          'longitude', 
          'absoluteAltitude', 
          'trueHeading', 
          'groundSpeed', 
          'trueAirspeed', 
          'climbOrDescentRate', 
          'hasWeightOnWheels', 
          'turnRate to degPerSec', 
          'cyclicPositionPitch', 
          'cyclicPositionRoll', 
          'collectivePosition', 
          'antiTorquePedalPosition', 
          'throttlePosition', 
          'e1Torque', 
          'e2Torque',
          'turnDirection',
          'Altdiff',
          'groundSpeedDiff'
          
]] # only capture columns that relevant into another dataframe

# Convert timestamp into year month day and hour in seconds 
# eliminate the milliseonds part in order to group them by seconds with an average
df2['datetime'] = df2.datetime.dt.strftime('%Y-%m-%d %H:%M:%S.%f').astype('datetime64[ns]')


# Group all row by seconds with a average all the datapoint into another dataframe
df3 = df2.groupby(by=["datetime"], dropna=False).mean().reset_index()

C:\Users\hrish\AppData\Local\Temp\ipykernel_5404\4229051035.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['datetime'] = df2.datetime.dt.strftime('%Y-%m-%d %H:%M:%S.%f').astype('datetime64[ns]')


In [14]:
df3['HoverOrNot'] = 0

df3.loc[(df3['datetime'].between('2022-06-30 14:28:13' , '2022-06-30 14:33:12' , inclusive='both')),['HoverOrNot']] = 1

df3.loc[(df3['datetime'].between('2022-06-30 14:34:10' , '2022-06-30 14:38:29' , inclusive='both')),['HoverOrNot']] = 1

df3.loc[(df3['datetime'].between('2022-06-30 14:40:10' , '2022-06-30 14:43:40' , inclusive='both')),['HoverOrNot']] = 1

df3.loc[(df3['datetime'].between('2022-06-30 14:46:00' , '2022-06-30 14:55:55' , inclusive='both')),['HoverOrNot']] = 1

df3.loc[(df3['datetime'].between('2022-06-30 14:57:10' , '2022-06-30 15:00:16' , inclusive='both')),['HoverOrNot']] = 1

df3.loc[(df3['datetime'].between('2022-06-30 15:01:20' , '2022-06-30 15:07:03' , inclusive='both')),['HoverOrNot']] = 1

df3.loc[(df3['datetime'].between('2022-06-30 15:07:38' , '2022-06-30 15:10:38' , inclusive='both')),['HoverOrNot']] = 1

df3.loc[(df3['datetime'].between('2022-06-30 15:12:02' , '2022-06-30 15:18:30' , inclusive='both')),['HoverOrNot']] = 1

df3.loc[(df3['datetime'].between('2022-06-30 15:20:18' , '2022-06-30 15:23:52' , inclusive='both')),['HoverOrNot']] = 1

df3.loc[(df3['datetime'].between('2022-06-30 15:25:30' , '2022-06-30 15:30:17' , inclusive='both')),['HoverOrNot']] = 1

df3.loc[(df3['datetime'].between('2022-06-30 15:36:45' , '2022-06-30 15:40:24' , inclusive='both')),['HoverOrNot']] = 1

df3.loc[(df3['datetime'].between('2022-06-30 15:42:28' , '2022-06-30 15:55:20' , inclusive='both')),['HoverOrNot']] = 1


# Combine with the new data

df3.loc[(df3['datetime'].between('2022-07-07 13:58:38' , '2022-07-07 14:02:54' , inclusive='both')),['HoverOrNot']] = 1

df3.loc[(df3['datetime'].between('2022-07-07 14:03:40' , '2022-07-07 14:11:19' , inclusive='both')),['HoverOrNot']] = 1

df3.loc[(df3['datetime'].between('2022-07-07 14:14:55' , '2022-07-07 14:21:03' , inclusive='both')),['HoverOrNot']] = 1

df3.loc[(df3['datetime'].between('2022-07-07 14:22:19' , '2022-07-07 14:23:18' , inclusive='both')),['HoverOrNot']] = 1

df3.loc[(df3['datetime'].between('2022-07-07 14:24:20' , '2022-07-07 14:35:00' , inclusive='both')),['HoverOrNot']] = 1

df3.loc[(df3['datetime'].between('2022-07-07 14:36:00' , '2022-07-07 14:45:24' , inclusive='both')),['HoverOrNot']] = 1

df3.loc[(df3['datetime'].between('2022-07-07 14:46:50' , '2022-07-07 14:47:20' , inclusive='both')),['HoverOrNot']] = 1

# Double check to see hasWeightOnWheels = 1, then set Hover Or Not = 0
df3.loc[(df3['hasWeightOnWheels'] == 1), ['HoverOrNot']] = 0
df5=df3
df5.head(5)

,datetime,latitude,longitude,absoluteAltitude,trueHeading,groundSpeed,trueAirspeed,climbOrDescentRate,hasWeightOnWheels,turnRate to degPerSec,...,cyclicPositionRoll,collectivePosition,antiTorquePedalPosition,throttlePosition,e1Torque,e2Torque,turnDirection,Altdiff,groundSpeedDiff,HoverOrNot
0,2022-06-30 14:27:53.216,20.904799,-156.424679,0.12,213.554,0.008887,0.024884,0.122,0.0,-0.204,...,39.330,3.465,12.395,0.0,-4.350,-4.350,0.000,0.0,0.000000,0
1,2022-06-30 14:27:53.234,20.904799,-156.424679,0.12,213.547,0.010072,0.030809,0.115,0.0,-0.201,...,37.337,3.465,12.395,0.0,-4.347,-4.347,0.000,0.0,0.000000,0
2,2022-06-30 14:27:53.251,20.904799,-156.424679,0.12,213.541,0.018959,0.041474,0.105,0.0,-0.197,...,35.317,3.465,12.395,0.0,-4.342,-4.342,0.000,0.0,0.000000,0
3,2022-06-30 14:27:53.271,20.904799,-156.424679,0.12,213.534,0.029032,0.053916,0.092,0.0,-0.192,...,33.689,3.465,12.395,0.0,-4.335,-4.335,0.000,0.0,0.000000,0
4,2022-06-30 14:27:53.293,20.904799,-156.424679,0.12,213.526,0.039104,0.066951,0.075,0.0,-0.186,...,32.547,3.465,12.395,0.0,-4.326,-4.326,-0.028,0.0,-0.030217,0


In [15]:
from pandas.plotting import scatter_matrix
from matplotlib import cm

feature_names = [
          'absoluteAltitude', 
          'trueHeading', 
          'groundSpeed', 
          'trueAirspeed', 
          'climbOrDescentRate', 
          'turnRate to degPerSec', 
          'cyclicPositionPitch', 
          'cyclicPositionRoll', 
          'collectivePosition', 
          'antiTorquePedalPosition', 
          'throttlePosition', 
          'e1Torque', 
          'e2Torque',
          'turnDirection',
          'Altdiff',
          'groundSpeedDiff',
        'HoverOrNot']               
                
                
x = df5[feature_names]
y = df5['HoverOrNot']
len(feature_names)

#https://www.analyticsvidhya.com/blog/2019/01/introduction-time-series-classification/?
#https://www.cis.fordham.edu/wisdm/dataset.php
#https://towardsdatascience.com/time-series-classification-for-human-activity-recognition-with-lstms-using-tensorflow-2-and-keras-b816431afdff

17

In [16]:
# convert dataframe to multiple arrays

def df_to_arr_by_sequence(df_input):
    check_hover=0
    previous=0
    rotorcraft_np_arr_main=[]
    rotorcraft_np_arr_target=[]
    for i in range(len(df_input)):
        if(df_input.loc[i,"HoverOrNot"]!=check_hover):
            data = df_input.iloc[previous:i]
            data = data.drop(columns=['HoverOrNot'])
            rotorcraft_np_arr_main.append(data)
            #rotorcraft_np_arr_target.append([check_hover] * (i-previous))
            rotorcraft_np_arr_target.append(check_hover)
            #print("hover before ", check_hover)
            check_hover=1 if check_hover == 0 else 0
            #print("hover after ", check_hover)
            previous=i
    return rotorcraft_np_arr_main,rotorcraft_np_arr_target

x_main,x_target = df_to_arr_by_sequence(x)
            


            
                                
        

In [17]:
x_main = np.asarray(x_main, dtype=object)
x_target = np.asarray(x_target, dtype=np.float32)
print(x_main.size, " ", x_target.size)
pd.Series(x_main).describe()
x_target[0]

46   46


0.0

In [18]:
from keras_preprocessing.sequence import pad_sequences
#X=pad_sequences(X, maxlen=len(feature_names), padding='post', dtype='float', truncating='post')
seq_len = 1000
final_seq=pad_sequences(x_main, maxlen=seq_len, padding='post', dtype='float', truncating='post')
model = Sequential()
model.add(LSTM(256, input_shape=(seq_len, len(feature_names)-1)))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 256)               279552    
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 279,809
Trainable params: 279,809
Non-trainable params: 0
_________________________________________________________________


In [19]:
type(final_seq)
X_train = final_seq[np.r_[0:int(len(final_seq)*0.8)],:]
X_test = x_target[np.r_[0:int(len(x_target)*0.8)]]
print(len(X_train)-1)
Y_train = final_seq[np.r_[int(len(final_seq)*0.8):(len(final_seq))],:]
Y_test = x_target[np.r_[int(len(x_target)*0.8):(len(x_target))]]
print(len(final_seq))
print(len(X_train)," ",len(Y_train))
print(len(X_test)," ",len(Y_test))
#np.ndarray[0 : len(x_target)*0.8, : ]
print(Y_train.shape)

35
46
36   10
36   10
(10, 1000, 16)


In [20]:

adam = Adam(lr=0.001)
chk = ModelCheckpoint('best_model.pkl', monitor='val_acc', save_best_only=True, mode='max', verbose=1)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.fit(X_train, X_test, epochs=200, batch_size=128, callbacks=[chk], validation_data=(Y_train,Y_test))


Epoch 1/200


C:\Users\hrish\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1/1 [==============================] - 6s 6s/step - loss: 0.6988 - accuracy: 0.6389 - val_loss: 0.6197 - val_accuracy: 0.6000
Epoch 2/200
1/1 [==============================] - 3s 3s/step - loss: 0.5835 - accuracy: 0.6944 - val_loss: 0.5828 - val_accuracy: 0.7000
Epoch 3/200
1/1 [==============================] - 3s 3s/step - loss: 0.4973 - accuracy: 0.7500 - val_loss: 0.5692 - val_accuracy: 0.7000
Epoch 4/200
1/1 [==============================] - 3s 3s/step - loss: 0.4296 - accuracy: 0.8611 - val_loss: 0.5611 - val_accuracy: 0.7000
Epoch 5/200
1/1 [==============================] - 3s 3s/step - loss: 0.3767 - accuracy: 0.8889 - val_loss: 0.5472 - val_accuracy: 0.7000
Epoch 6/200
1/1 [==============================] - 3s 3s/step - loss: 0.3383 - accuracy: 0.9167 - val_loss: 0.5402 - val_accuracy: 0.8000
Epoch 7/200
1/1 [==============================] - 3s 3s/step - loss: 0.3070 - accuracy: 0.9444 - val_loss: 0.5769 - val_accuracy: 0.8000
Epoch 8/200
1/1 [=============================

Epoch 29/200
1/1 [==============================] - 3s 3s/step - loss: 0.0920 - accuracy: 0.9722 - val_loss: 0.4390 - val_accuracy: 0.8000
Epoch 30/200
1/1 [==============================] - 3s 3s/step - loss: 0.0880 - accuracy: 0.9722 - val_loss: 0.4315 - val_accuracy: 0.8000
Epoch 31/200
1/1 [==============================] - 3s 3s/step - loss: 0.0850 - accuracy: 0.9722 - val_loss: 0.4196 - val_accuracy: 0.8000
Epoch 32/200
1/1 [==============================] - 3s 3s/step - loss: 0.0824 - accuracy: 0.9722 - val_loss: 0.4151 - val_accuracy: 0.8000
Epoch 33/200
1/1 [==============================] - 3s 3s/step - loss: 0.0820 - accuracy: 0.9722 - val_loss: 0.4129 - val_accuracy: 0.8000
Epoch 34/200
1/1 [==============================] - 3s 3s/step - loss: 0.0799 - accuracy: 0.9722 - val_loss: 0.4094 - val_accuracy: 0.8000
Epoch 35/200
1/1 [==============================] - 3s 3s/step - loss: 0.0780 - accuracy: 0.9722 - val_loss: 0.4065 - val_accuracy: 0.8000
Epoch 36/200
1/1 [=========

Epoch 57/200
1/1 [==============================] - 3s 3s/step - loss: 0.0593 - accuracy: 0.9722 - val_loss: 0.3626 - val_accuracy: 0.8000
Epoch 58/200
1/1 [==============================] - 3s 3s/step - loss: 0.0591 - accuracy: 0.9722 - val_loss: 0.3600 - val_accuracy: 0.8000
Epoch 59/200
1/1 [==============================] - 3s 3s/step - loss: 0.0588 - accuracy: 0.9722 - val_loss: 0.3618 - val_accuracy: 0.8000
Epoch 60/200
1/1 [==============================] - 3s 3s/step - loss: 0.0586 - accuracy: 0.9722 - val_loss: 0.3503 - val_accuracy: 0.8000
Epoch 61/200
1/1 [==============================] - 3s 3s/step - loss: 0.0583 - accuracy: 0.9722 - val_loss: 0.3487 - val_accuracy: 0.8000
Epoch 62/200
1/1 [==============================] - 3s 3s/step - loss: 0.0581 - accuracy: 0.9722 - val_loss: 0.3564 - val_accuracy: 0.8000
Epoch 63/200
1/1 [==============================] - 3s 3s/step - loss: 0.0578 - accuracy: 0.9722 - val_loss: 0.3557 - val_accuracy: 0.8000
Epoch 64/200
1/1 [=========

Epoch 85/200
1/1 [==============================] - 3s 3s/step - loss: 0.0556 - accuracy: 0.9722 - val_loss: 0.3357 - val_accuracy: 0.8000
Epoch 86/200
1/1 [==============================] - 3s 3s/step - loss: 0.0555 - accuracy: 0.9722 - val_loss: 0.3355 - val_accuracy: 0.8000
Epoch 87/200
1/1 [==============================] - 3s 3s/step - loss: 0.0554 - accuracy: 0.9722 - val_loss: 0.3354 - val_accuracy: 0.8000
Epoch 88/200
1/1 [==============================] - 3s 3s/step - loss: 0.0554 - accuracy: 0.9722 - val_loss: 0.3352 - val_accuracy: 0.8000
Epoch 89/200
1/1 [==============================] - 4s 4s/step - loss: 0.0553 - accuracy: 0.9722 - val_loss: 0.3348 - val_accuracy: 0.8000
Epoch 90/200
1/1 [==============================] - 4s 4s/step - loss: 0.0553 - accuracy: 0.9722 - val_loss: 0.3344 - val_accuracy: 0.8000
Epoch 91/200
1/1 [==============================] - 3s 3s/step - loss: 0.0552 - accuracy: 0.9722 - val_loss: 0.3339 - val_accuracy: 0.8000
Epoch 92/200
1/1 [=========

1/1 [==============================] - 3s 3s/step - loss: 0.0544 - accuracy: 0.9722 - val_loss: 0.3282 - val_accuracy: 0.8000
Epoch 113/200
1/1 [==============================] - 3s 3s/step - loss: 0.0543 - accuracy: 0.9722 - val_loss: 0.3281 - val_accuracy: 0.8000
Epoch 114/200
1/1 [==============================] - 3s 3s/step - loss: 0.0543 - accuracy: 0.9722 - val_loss: 0.3281 - val_accuracy: 0.8000
Epoch 115/200
1/1 [==============================] - 3s 3s/step - loss: 0.0543 - accuracy: 0.9722 - val_loss: 0.3281 - val_accuracy: 0.8000
Epoch 116/200
1/1 [==============================] - 3s 3s/step - loss: 0.0543 - accuracy: 0.9722 - val_loss: 0.3280 - val_accuracy: 0.8000
Epoch 117/200
1/1 [==============================] - 3s 3s/step - loss: 0.0542 - accuracy: 0.9722 - val_loss: 0.3280 - val_accuracy: 0.8000
Epoch 118/200
1/1 [==============================] - 3s 3s/step - loss: 0.0542 - accuracy: 0.9722 - val_loss: 0.3279 - val_accuracy: 0.8000
Epoch 119/200
1/1 [===============

1/1 [==============================] - 4s 4s/step - loss: 0.0539 - accuracy: 0.9722 - val_loss: 0.3236 - val_accuracy: 0.8000
Epoch 141/200
1/1 [==============================] - 3s 3s/step - loss: 0.0539 - accuracy: 0.9722 - val_loss: 0.3235 - val_accuracy: 0.8000
Epoch 142/200
1/1 [==============================] - 4s 4s/step - loss: 0.0538 - accuracy: 0.9722 - val_loss: 0.3234 - val_accuracy: 0.8000
Epoch 143/200
1/1 [==============================] - 5s 5s/step - loss: 0.0538 - accuracy: 0.9722 - val_loss: 0.3233 - val_accuracy: 0.8000
Epoch 144/200
1/1 [==============================] - 4s 4s/step - loss: 0.0538 - accuracy: 0.9722 - val_loss: 0.3232 - val_accuracy: 0.8000
Epoch 145/200
1/1 [==============================] - 4s 4s/step - loss: 0.0538 - accuracy: 0.9722 - val_loss: 0.3232 - val_accuracy: 0.8000
Epoch 146/200
1/1 [==============================] - 4s 4s/step - loss: 0.0538 - accuracy: 0.9722 - val_loss: 0.3232 - val_accuracy: 0.8000
Epoch 147/200
1/1 [===============

1/1 [==============================] - 3s 3s/step - loss: 0.0537 - accuracy: 0.9722 - val_loss: 0.3218 - val_accuracy: 0.8000
Epoch 168/200
1/1 [==============================] - 3s 3s/step - loss: 0.0537 - accuracy: 0.9722 - val_loss: 0.3218 - val_accuracy: 0.8000
Epoch 169/200
1/1 [==============================] - 3s 3s/step - loss: 0.0537 - accuracy: 0.9722 - val_loss: 0.3217 - val_accuracy: 0.8000
Epoch 170/200
1/1 [==============================] - 3s 3s/step - loss: 0.0537 - accuracy: 0.9722 - val_loss: 0.3216 - val_accuracy: 0.8000
Epoch 171/200
1/1 [==============================] - 3s 3s/step - loss: 0.0537 - accuracy: 0.9722 - val_loss: 0.3216 - val_accuracy: 0.8000
Epoch 172/200
1/1 [==============================] - 3s 3s/step - loss: 0.0537 - accuracy: 0.9722 - val_loss: 0.3215 - val_accuracy: 0.8000
Epoch 173/200
1/1 [==============================] - 3s 3s/step - loss: 0.0537 - accuracy: 0.9722 - val_loss: 0.3215 - val_accuracy: 0.8000
Epoch 174/200
1/1 [===============

1/1 [==============================] - 3s 3s/step - loss: 0.0535 - accuracy: 0.9722 - val_loss: 0.3195 - val_accuracy: 0.8000
Epoch 196/200
1/1 [==============================] - 3s 3s/step - loss: 0.0535 - accuracy: 0.9722 - val_loss: 0.3194 - val_accuracy: 0.8000
Epoch 197/200
1/1 [==============================] - 3s 3s/step - loss: 0.0535 - accuracy: 0.9722 - val_loss: 0.3192 - val_accuracy: 0.8000
Epoch 198/200
1/1 [==============================] - 3s 3s/step - loss: 0.0535 - accuracy: 0.9722 - val_loss: 0.3189 - val_accuracy: 0.8000
Epoch 199/200
1/1 [==============================] - 3s 3s/step - loss: 0.0535 - accuracy: 0.9722 - val_loss: 0.3187 - val_accuracy: 0.8000
Epoch 200/200
1/1 [==============================] - 3s 3s/step - loss: 0.0535 - accuracy: 0.9722 - val_loss: 0.3185 - val_accuracy: 0.8000


In [21]:
#model = load_model('best_model.pkl')

from sklearn.metrics import accuracy_score
test_preds = (model.predict(Y_train) > 0.5).astype("int32")
accuracy_score(Y_test, test_preds)

1/1 [==============================] - 1s 687ms/step


0.8